In [1]:
import pandas as pd

%load_ext autoreload
%autoreload 2

# Extra options
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
%matplotlib agg

import logging
logging.basicConfig(level=logging.INFO)

import os
import osmnx as ox
import sys
import importlib
import numpy as np
import matplotlib.pyplot as plt
from shapely import wkt 
import geopandas as gpd
sys.path.insert(0, os.path.abspath('../../'))

In [2]:
with open('../../keys/mapbox_key.txt', 'r') as f:
    mapbox_key = f.readline()

In [3]:
from osmnx_network_extract.network_code import return_crs
from osmnx_network_extract.network_code import NetworkCode
from visualise.customer_plots import return_scatter_layer, color_df, return_arc_layer
import pydeck as pdk

In [4]:
prep_network = pd.read_csv('test/cit_punggol_internal_50m_buffer_contain.csv')

In [5]:
prep_network

,u,v,key,osmid,lanes,highway,service,oneway,length,geometry,name,maxspeed,...,ref,access,junction,width,arc_id,arc_id_orig,u_orig,v_orig,arc_id_ordered,arc_id_ordered_orig,parallel,Name
0,4604559376,4488368046,0,"[753159712, 639706205, 753159711]",4,tertiary,NaN,False,59.830,"LINESTRING (103.8777916 1.3228068, 103.877663 ...",Kallang Way,50,...,NaN,NaN,NaN,NaN,4604559376-4488368046,4604559376-4488368046-0,4.604559e+09,4.488368e+09,4488368046-4604559376,4488368046-4604559376,False,Sub-sector A - City
1,4604559376,5683821101,0,"[666595169, 666595170, 644399321]",2,residential,NaN,False,51.201,"LINESTRING (103.8777916 1.3228068, 103.8777027...",Kallang Way 2A,50,...,NaN,NaN,NaN,NaN,4604559376-5683821101,4604559376-5683821101-0,4.604559e+09,5.683821e+09,4604559376-5683821101,4604559376-5683821101,False,Sub-sector A - City
2,4604559376,4488368055,0,"[764039529, 764039530, 639711921]",4,tertiary,NaN,False,20.401,"LINESTRING (103.8777916 1.3228068, 103.8778413...",Kallang Way,50,...,NaN,NaN,NaN,NaN,4604559376-4488368055,4604559376-4488368055-0,4.604559e+09,4.488368e+09,4488368055-4604559376,4488368055-4604559376,False,Sub-sector A - City
3,385613932,366653047,0,"[383372420, 693663613]",3,motorway,NaN,True,1391.466,"LINESTRING (103.8766693 1.2973415, 103.8767555...",Kallang-Paya Lebar Expressway,80,...,KPE,NaN,NaN,NaN,385613932-366653047,385613932-366653047-0,3.856139e+08,3.666530e+08,385613932-366653047,385613932-366653047-0,False,Sub-sector A - City
4,4604821646,243230385,0,637455171,3,tertiary,NaN,False,35.944,"LINESTRING (103.8991205 1.3264995, 103.8988063...",Ubi Avenue 1,50,...,NaN,NaN,NaN,NaN,4604821646-243230385,4604821646-243230385-0,4.604822e+09,2.432304e+08,243230385-4604821646,243230385-4604821646,False,Sub-sector A - City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26358,77307331507,773073539227,1,543735156,1,service,NaN,True,45.029,"LINESTRING (103.9026901 1.385734, 103.9026142 ...",NaN,NaN,...,NaN,NaN,NaN,NaN,77307331507-773073539227,5256240692-5256240685-1,5.256241e+09,5.256241e+09,77307331507-773073539227,5256240692-5256240685-1,True,Sub-sector B - Punggol
26359,77307331877,773073539597,1,"[735217973, 543352142]",1,service,parking_aisle,True,92.503,"LINESTRING (103.8930374 1.3724717, 103.89303 1...",NaN,NaN,...,NaN,NaN,NaN,NaN,77307331877-773073539597,5252571177-5252571170-1,5.252571e+09,5.252571e+09,77307331877-773073539597,5252571177-5252571170-1,True,Sub-sector B - Punggol
26360,77307332361,773073540081,0,"[643446253, 599126893, 599126895]",1,service,NaN,True,216.760,"LINESTRING (103.886293 1.3441932, 103.8863292 ...",NaN,NaN,...,NaN,NaN,NaN,NaN,77307332361-773073540081,5701100079-5701100097-0,5.701100e+09,5.701100e+09,77307332361-773073540081,5701100079-5701100097-0,True,Sub-sector B - Punggol
26361,77307332362,773073540082,1,759531828,1,service,NaN,True,36.106,"LINESTRING (103.886293 1.3441932, 103.8863419 ...",NaN,NaN,...,NaN,NaN,NaN,NaN,77307332362-773073540082,5701100079-5701100097-1,5.701100e+09,5.701100e+09,77307332362-773073540082,5701100079-5701100097-1,True,Sub-sector B - Punggol


In [6]:
geocode_network = NetworkCode(prep_network, 'SG', xy_converted=False)

gdf_customers = geocode_network.create_nodes_gdf(geocode_network.df_network)

gdf_customers['lat'] = gdf_customers['y']  + np.random.uniform(low=-0.001, high=0.001, size=gdf_customers.shape[0])
gdf_customers['lon'] = gdf_customers['x']  + np.random.uniform(low=-0.001, high=0.001, size=gdf_customers.shape[0])

INFO:root:Extracting nodes from network
INFO:root:Extracting nodes from network
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


## Full examples auto

In [7]:
geocode_network2 = NetworkCode(prep_network, 'SG')
geocode_network2.convert_direct()

temp_customers = gdf_customers.copy()
temp_customers = temp_customers[['lat', 'lon']]
temp_customers['u_id'] = temp_customers.index
temp_customers = temp_customers.reset_index(drop=True)
geocode_network2.add_customers(temp_customers)
geocode_network2.find_nearest_collection_point()

df_customer_fit = geocode_network2.df_collection_points
plt_map = df_customer_fit[['lon', 'lat', 'lon_collect', 'lat_collect', 'dist_collect', 'arc_collect']].copy()
scatter_orig = return_scatter_layer(plt_map, 'original', 'lat', 'lon', [250, 0, 0, 250])
scatter_new = return_scatter_layer(plt_map, 'snapped', 'lat_collect', 'lon_collect', [250, 250, 0, 250])
arcs = return_arc_layer(plt_map, 'arcs', 'lat', 'lon', 'lat_collect', 'lon_collect', [250, 0, 0, 250], [250, 250, 0, 250])

viewport = pdk.data_utils.compute_view(points=plt_map[['lon', 'lat']], view_proportion=0.95)

r = pdk.Deck(layers=[scatter_orig, scatter_new, arcs], 
              initial_view_state=viewport, 
              mapbox_key = mapbox_key, 
              height = 1250,
              width = 1250
             )
r.show()

INFO:root:Convert network to x-y
INFO:root:Extracting nodes from network
INFO:root:Removing opposing network arcs (only need one).
INFO:root:Arcs removed: 7938.
INFO:root:Adding 14919 customers.
INFO:root:Find nearest arc
INFO:root:Number of arcs 18425 number of points 14919
INFO:root:Find closest assigned end point
INFO:root:Find closest point on assigned arc
INFO:root:Find best collection point


DeckGLWidget(height=1250, json_input='{"initialViewState": {"latitude": 1.347388999971771, "longitude": 103.88…

## Full examples residential

In [8]:
geocode_network2 = NetworkCode(prep_network, 'SG')
geocode_network2.convert_direct()

temp_customers = gdf_customers.copy()
temp_customers = temp_customers[['lat', 'lon']]
temp_customers['u_id'] = temp_customers.index
temp_customers = temp_customers.reset_index(drop=True)
geocode_network2.add_customers(temp_customers)
geocode_network2.filter_network(['residential'])
geocode_network2.find_nearest_collection_point()

INFO:root:Convert network to x-y
INFO:root:Extracting nodes from network
INFO:root:Removing opposing network arcs (only need one).
INFO:root:Arcs removed: 7938.
INFO:root:Adding 14919 customers.
INFO:root:Filtering network on: highway
INFO:root:Arcs removed: 14133
INFO:root:Find nearest arc
INFO:root:Number of arcs 4292 number of points 14919
INFO:root:Find closest assigned end point
INFO:root:Find closest point on assigned arc
INFO:root:Find best collection point


In [9]:
df_customer_fit = geocode_network2.df_collection_points
plt_map = df_customer_fit[['lon', 'lat', 'lon_collect', 'lat_collect', 'dist_collect', 'arc_collect']].copy()
scatter_orig = return_scatter_layer(plt_map, 'original', 'lat', 'lon', [250, 0, 0, 250])
scatter_new = return_scatter_layer(plt_map, 'snapped', 'lat_collect', 'lon_collect', [250, 250, 0, 250])
arcs = return_arc_layer(plt_map, 'arcs', 'lat', 'lon', 'lat_collect', 'lon_collect', [250, 0, 0, 250], [250, 250, 0, 250])

viewport = pdk.data_utils.compute_view(points=plt_map[['lon', 'lat']], view_proportion=0.95)

r = pdk.Deck(layers=[scatter_orig, scatter_new, arcs], 
              initial_view_state=viewport, 
              mapbox_key = mapbox_key, 
              height = 1250,
              width = 1250
             )
r.show()

DeckGLWidget(height=1250, json_input='{"initialViewState": {"latitude": 1.3473889999717747, "longitude": 103.8…